This model combines the CNN with a transformer and includes spatial features of the protein. It also incorporates the protein labels.

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import glob
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict

DEBUG_MODE = False  # Set to False to disable debug messages
total_samples = 200000 #set the number of samples you want to test. Total number of samples is 1,079,034


# Set seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Output directory for intermediate files
OUTPUT_DIR = os.path.join(os.getcwd(), "Results")
os.makedirs(OUTPUT_DIR, exist_ok=True)


# Debugging Utility Function
def debug_tensor_shape(tensor, name):
    """Utility function to debug and print tensor shape."""
    if DEBUG_MODE:
        print(f"[DEBUG] {name} shape: {tensor.shape}")
    return tensor


# Helper Functions to Process RNA and Protein Data
def save_to_csv(data, filename):
    filepath = os.path.join(OUTPUT_DIR, filename)
    if isinstance(data, np.ndarray):
        pd.DataFrame(data).to_csv(filepath, index=False, header=False)
    elif isinstance(data, pd.DataFrame):
        data.to_csv(filepath, index=False)
    print(f"[INFO] Saved {filename} to {OUTPUT_DIR}")

def load_protein_data(protein_file_path):
    protein_data = pd.read_csv(protein_file_path)
    protein_label = protein_data["Protein_Label"].iloc[0]
    protein_data = protein_data.drop(columns=["Protein_Label"], errors="ignore")
    protein_data = protein_data.apply(pd.to_numeric, errors="coerce").dropna(axis=1).values
    return protein_data, int(protein_label)

def load_rna_data(rna_file_path):
    rna_data = pd.read_csv(rna_file_path)
    rna_data = rna_data.drop(columns=["Sequence_Name", "Position"], errors="ignore")
    rna_data = rna_data.apply(pd.to_numeric, errors="coerce").dropna(axis=1).values
    num_samples = rna_data.shape[0] // 101
    rna_samples = rna_data[:num_samples * 101].reshape(num_samples, 101, -1)
    rna_features = rna_samples[:, :, :-1]
    rna_binding = rna_samples[:, 0, -1]
    return rna_features, rna_binding


def process_data(base_folder_path, total_samples):
    rna_data_list = []
    protein_data_list = []
    binding_labels = []
    protein_labels = []
    rna_folders = []

    class_counts = defaultdict(int)
    max_per_class = total_samples // 17

    for root, _, _ in os.walk(base_folder_path):  # Removed `dirs` and `files`
        if "Protein data" in root:
            protein_folder = root
            rna_folder = os.path.dirname(protein_folder)

            protein_file = glob.glob(os.path.join(protein_folder, "*_Concatenated_Encoding_Matrix.csv"))
            if not protein_file:
                continue
            protein_file = protein_file[0]

            rna_file = os.path.join(rna_folder, "RNA_sequences_combined_stacked.csv")
            if not os.path.exists(rna_file):
                continue

            protein_data, protein_label = load_protein_data(protein_file)
            rna_data, rna_binding = load_rna_data(rna_file)

            protein_label -= 1
            if class_counts[protein_label] >= max_per_class:
                continue

            sample_size = min(len(rna_data), max_per_class - class_counts[protein_label])
            selected_indices = random.sample(range(len(rna_data)), sample_size)
            rna_data_sample = rna_data[selected_indices]

            for rna in rna_data_sample:  # Removed unused `binding`
                if class_counts[protein_label] >= max_per_class:
                    break
                rna_data_list.append(rna)
                protein_data_list.append(np.expand_dims(protein_data, axis=0))
                binding_labels.append(1)  # Assumes binding values are all 1; adjust as needed.
                protein_labels.append(protein_label)
                rna_folders.append(rna_folder)
                class_counts[protein_label] += 1

    if not rna_data_list or not protein_data_list:
        raise ValueError("[ERROR] No RNA or Protein data was successfully loaded.")

    rna_data_array = np.array(rna_data_list, dtype=np.float32)
    protein_data_array = np.concatenate(protein_data_list, axis=0)
    binding_labels_array = np.array(binding_labels, dtype=np.float32)
    protein_labels_array = tf.keras.utils.to_categorical(protein_labels, num_classes=17)

    return rna_data_array, protein_data_array, binding_labels_array, protein_labels_array, rna_folders

# Model Creation
def create_hybrid_model(rna_input_shape, protein_input_shape):
    # RNA Feature Extraction
    rna_input = layers.Input(shape=(101, 8), name="RNA_Input")
    print(f"[DEBUG] Initial RNA Input Shape: {rna_input.shape}")
    x_rna = layers.Conv1D(64, 3, activation="relu")(rna_input)
    x_rna = layers.MaxPooling1D(2)(x_rna)
    x_rna = layers.Conv1D(128, 3, activation="relu")(x_rna)
    x_rna = layers.MaxPooling1D(2)(x_rna)
    x_rna = layers.Flatten()(x_rna)

    # Protein Feature Extraction
    protein_input = layers.Input(shape=(1003, 27), name="Protein_Input")
    print(f"[DEBUG] Initial Protein Input Shape: {protein_input.shape}")
    x_protein = layers.Conv1D(64, 3, activation="relu")(protein_input)
    x_protein = layers.MaxPooling1D(5)(x_protein)
    x_protein = layers.Conv1D(128, 3, activation="relu")(x_protein)
    x_protein = layers.MaxPooling1D(2)(x_protein)
    x_protein = layers.Flatten()(x_protein)

    # Combine Features
    combined = layers.Concatenate()([x_rna, x_protein])
    debug_tensor_shape(combined, "Combined Features for Transformer")
    combined = layers.Reshape((1, combined.shape[-1]))(combined)
    debug_tensor_shape(combined, "Reshaped Combined Features for Transformer")
    combined = layers.MultiHeadAttention(num_heads=4, key_dim=64)(combined, combined)
    debug_tensor_shape(combined, "Transformer Output")
    combined = layers.GlobalAveragePooling1D()(combined)
    debug_tensor_shape(combined, "Global Pooled Features")

    # Separate Path for Binding Prediction
    binding_path = layers.Dense(128, activation="relu")(combined)
    debug_tensor_shape(binding_path, "binding path output")
    binding_output = layers.Dense(1, activation="sigmoid", name="Binding_Prediction")(binding_path)

    # Separate Path for Protein Prediction
    protein_path = layers.Dense(128, activation="relu")(combined)
    debug_tensor_shape(protein_path, "Protein path output")
    protein_output = layers.Dense(17, activation="softmax", name="Protein_Prediction")(protein_path)

    # Model
    model = models.Model(inputs=[rna_input, protein_input], outputs=[binding_output, protein_output])
    model.compile(
        optimizer="adam",
        loss={
            "Binding_Prediction": "binary_crossentropy",
            "Protein_Prediction": "categorical_crossentropy",
        },
        metrics={
            "Binding_Prediction": "accuracy",
            "Protein_Prediction": "accuracy",
        },
    )
    return model

# Main
if __name__ == "__main__":
    script_dir = os.getcwd()
    base_folder_path = os.path.join(script_dir, "Data", "datasets", "clip")

    rna_data, protein_data, binding_labels, protein_labels, rna_folders = process_data(base_folder_path, total_samples)
    indices = np.arange(len(rna_data))
    train_indices, test_indices = train_test_split(
        indices, test_size=0.2, stratify=protein_labels.argmax(axis=1), random_state=42
    )
    rna_train, rna_test = rna_data[train_indices], rna_data[test_indices]
    protein_train, protein_test = protein_data[train_indices], protein_data[test_indices]
    binding_train, binding_test = binding_labels[train_indices], binding_labels[test_indices]
    protein_label_train, protein_label_test = protein_labels[train_indices], protein_labels[test_indices]

    rna_input_shape = rna_train.shape[1:]
    protein_input_shape = protein_train.shape[1:]
    model = create_hybrid_model(rna_input_shape, protein_input_shape)

    print(f"[DEBUG] RNA Train Input Shape: {rna_train.shape}")
    print(f"[DEBUG] Protein Train Input Shape: {protein_train.shape}")

    # Perform a random split
    train_rna, val_rna, train_protein, val_protein, train_binding, val_binding, train_labels, val_labels = train_test_split(
        rna_train, protein_train, binding_train, protein_label_train, test_size=0.2, random_state=42
    )

    # Train with manually split validation set
    model.fit(
        [train_rna, train_protein],
        {"Binding_Prediction": train_binding, "Protein_Prediction": train_labels},
        validation_data=([val_rna, val_protein], {"Binding_Prediction": val_binding, "Protein_Prediction": val_labels}),
        epochs=3,
        batch_size=32
    )


    binding_predictions = model.predict([rna_test, protein_test])[0]  # Binding predictions
    protein_predictions = model.predict([rna_test, protein_test])[1]  # Protein class likelihoods

    # Prepare Metadata for Outputs
    rna_folders_test = [rna_folders[i] for i in test_indices]  # Get folder names for test data
    rna_numbers_test = np.arange(len(rna_test))  # Generate RNA numbers (you may have a specific mapping)
    binding_ground_truth = binding_test  # Ground truth for binding
    protein_ground_truth = np.argmax(protein_label_test, axis=1)  # Ground truth protein classes

    # Convert Predictions to Required Formats
    binding_pred_binary = (binding_predictions > 0.5).astype(int)  # Convert to binary
    protein_pred_classes = np.argmax(protein_predictions, axis=1)  # Predicted protein classes

    # First File: Core Predictions and Metadata
    core_output_data = pd.DataFrame({
        "Folder_Name": rna_folders_test,
        "RNA_Number": rna_numbers_test,
        "Binding_Ground_Truth": binding_ground_truth,
        "Binding_Prediction": binding_pred_binary.flatten(),
        "Protein_Ground_Class": protein_ground_truth,
        "Protein_Predicted_Class": protein_pred_classes
    })
    core_output_file = os.path.join(OUTPUT_DIR, "core_predictions.csv")
    core_output_data.to_csv(core_output_file, index=False)
    print(f"[INFO] Core predictions saved to {core_output_file}")

    # Second File: Protein Likelihoods
    likelihood_output_data = pd.DataFrame({
        "Folder_Name": rna_folders_test,
        "RNA_Number": rna_numbers_test,
    })

    # Add Likelihoods for Each Protein Class
    for class_index in range(protein_predictions.shape[1]):  # Loop over all 17 classes
        likelihood_output_data[f"Protein_Class_{class_index}_Likelihood"] = protein_predictions[:, class_index]

    likelihood_output_file = os.path.join(OUTPUT_DIR, "protein_likelihoods.csv")
    likelihood_output_data.to_csv(likelihood_output_file, index=False)
    print(f"[INFO] Protein likelihoods saved to {likelihood_output_file}")

  warnings.warn(
2024-11-27 13:46:49.857312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 177s 44ms/step - Binding_Prediction_accuracy: 0.9996 - Binding_Prediction_loss: 0.0281 - Protein_Prediction_accuracy: 0.0871 - Protein_Prediction_loss: 5.0269 - loss: 5.0549 - val_Binding_Prediction_accuracy: 1.0000 - val_Binding_Prediction_loss: 6.6461e-06 - val_Protein_Prediction_accuracy: 0.5893 - val_Protein_Prediction_loss: 1.3094 - val_loss: 1.3094
Epoch 2/3
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 170s 43ms/step - Binding_Prediction_accuracy: 0.9998 - Binding_Prediction_loss: 0.3687 - Protein_Prediction_accuracy: 0.0754 - Protein_Prediction_loss: 4.8736 - loss: 5.2423 - val_Binding_Prediction_accuracy: 1.0000 - val_Binding_Prediction_loss: 0.0000e+00 - val_Protein_Prediction_accuracy: 0.0581 - val_Protein_Prediction_loss: 2.8334 - val_loss: 2.8334
Epoch 3/3
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 174s 43ms/step - Binding_Prediction_accuracy: 1.0000 - Binding_Prediction_loss: 0.0000e+00 - Protein_Prediction_accuracy: 0.0581 - Protein_Prediction_loss: 2.8658 - loss: 2.8658 - val_Binding_Prediction_accuracy: 1.0000 - val_Binding_Prediction_loss: 0.0000e+00 - val_Protein_Prediction_accuracy: 0.0581 - val_Protein_Prediction_loss: 2.8334 - val_loss: 2.8334
   1/1250 ━━━━━━━━━━━━━━━━━━━━ 3:37 174ms/step
/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (32, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step

Although the previous model showed high accuracy prediction for both binding (1.0000) and protein prediction (0.7635) on a lower dataset (1,000). It failed to do so for bigger data sets of 100,000 samples as the binding prediction was  for binding prediction and    for protein prediction. This means that the model could be overfitting for the binding prediction and therefore driving the combined loss to increase the protein prediction. In order to improve the prediction for both models a balanced dataset was tested.

Additionally, even though the validation accuracy for binding protein was as high as the testing accuracy  (.98 -1.00) the validation accuracy for protein prediction was extrmely low with 0.0581.

As the total data set has

- **Binding Samples**: 295,375
- **Non-Binding Samples**: 783,659

The model is biased (overfitting) towards the non-binding samples so by having the same amount of binding samples vs non binding samples we would increase the model's accuracy.

model with balanced data, head = 4 and no positional emmbedings

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import glob
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict

DEBUG_MODE = False  # Set to False to disable debug messages
total_samples = 20000 #set the number of samples you want to test. Total number of samples is 1,079,034


# Set seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Output directory for intermediate files
OUTPUT_DIR = os.path.join(os.getcwd(), "Intermediate_steps")
os.makedirs(OUTPUT_DIR, exist_ok=True)


# Debugging Utility Function
def debug_tensor_shape(tensor, name):
    """Utility function to debug and print tensor shape."""
    if DEBUG_MODE:
        print(f"[DEBUG] {name} shape: {tensor.shape}")
    return tensor


# Helper Functions to Process RNA and Protein Data
def save_to_csv(data, filename):
    filepath = os.path.join(OUTPUT_DIR, filename)
    if isinstance(data, np.ndarray):
        pd.DataFrame(data).to_csv(filepath, index=False, header=False)
    elif isinstance(data, pd.DataFrame):
        data.to_csv(filepath, index=False)
    print(f"[INFO] Saved {filename} to {OUTPUT_DIR}")

def load_protein_data(protein_file_path):
    protein_data = pd.read_csv(protein_file_path)
    protein_label = protein_data["Protein_Label"].iloc[0]
    protein_data = protein_data.drop(columns=["Protein_Label"], errors="ignore")
    protein_data = protein_data.apply(pd.to_numeric, errors="coerce").dropna(axis=1).values
    return protein_data, int(protein_label)

def load_rna_data(rna_file_path):
    rna_data = pd.read_csv(rna_file_path)
    rna_data = rna_data.drop(columns=["Sequence_Name", "Position"], errors="ignore")
    rna_data = rna_data.apply(pd.to_numeric, errors="coerce").dropna(axis=1).values
    num_samples = rna_data.shape[0] // 101
    rna_samples = rna_data[:num_samples * 101].reshape(num_samples, 101, -1)
    rna_features = rna_samples[:, :, :-1]
    rna_binding = rna_samples[:, 0, -1]
    return rna_features, rna_binding

def process_data(base_folder_path, total_samples):
    rna_data_list = []
    protein_data_list = []
    binding_labels = []
    protein_labels = []
    rna_folders = []

    class_counts = defaultdict(int)
    max_per_class = total_samples // 34  # 17 classes × 2 (binding vs non-binding)

    for root, _, _ in os.walk(base_folder_path):  # Removed `dirs` and `files`
        if "Protein data" in root:
            protein_folder = root
            rna_folder = os.path.dirname(protein_folder)

            protein_file = glob.glob(os.path.join(protein_folder, "*_Concatenated_Encoding_Matrix.csv"))
            if not protein_file:
                continue
            protein_file = protein_file[0]

            rna_file = os.path.join(rna_folder, "RNA_sequences_combined_stacked.csv")
            if not os.path.exists(rna_file):
                continue

            protein_data, protein_label = load_protein_data(protein_file)
            rna_data, rna_binding = load_rna_data(rna_file)

            protein_label -= 1
            for i, rna in enumerate(rna_data):
                binding_label = rna_binding[i]

                # Enforce class balancing
                class_key = (protein_label, binding_label)
                if class_counts[class_key] >= max_per_class:
                    continue

                rna_data_list.append(rna)
                protein_data_list.append(np.expand_dims(protein_data, axis=0))
                binding_labels.append(binding_label)
                protein_labels.append(protein_label)
                rna_folders.append(rna_folder)
                class_counts[class_key] += 1

                # Stop if we have enough total samples
                if sum(class_counts.values()) >= total_samples:
                    break

    if not rna_data_list or not protein_data_list:
        raise ValueError("[ERROR] No RNA or Protein data was successfully loaded.")

    # Convert to arrays
    rna_data_array = np.array(rna_data_list, dtype=np.float32)
    protein_data_array = np.concatenate(protein_data_list, axis=0)
    binding_labels_array = np.array(binding_labels, dtype=np.float32)
    protein_labels_array = tf.keras.utils.to_categorical(protein_labels, num_classes=17)

    return rna_data_array, protein_data_array, binding_labels_array, protein_labels_array, rna_folders


# Model Creation
def create_hybrid_model(rna_input_shape, protein_input_shape):
    # RNA Feature Extraction
    rna_input = layers.Input(shape=(101, 8), name="RNA_Input")
    print(f"[DEBUG] Initial RNA Input Shape: {rna_input.shape}")
    x_rna = layers.Conv1D(64, 3, activation="relu")(rna_input)
    x_rna = layers.MaxPooling1D(2)(x_rna)
    x_rna = layers.Conv1D(128, 3, activation="relu")(x_rna)
    x_rna = layers.MaxPooling1D(2)(x_rna)
    x_rna = layers.Flatten()(x_rna)

    # Protein Feature Extraction
    protein_input = layers.Input(shape=(1003, 27), name="Protein_Input")
    print(f"[DEBUG] Initial Protein Input Shape: {protein_input.shape}")
    x_protein = layers.Conv1D(64, 3, activation="relu")(protein_input)
    x_protein = layers.MaxPooling1D(5)(x_protein)
    x_protein = layers.Conv1D(128, 3, activation="relu")(x_protein)
    x_protein = layers.MaxPooling1D(2)(x_protein)
    x_protein = layers.Flatten()(x_protein)

    # Combine Features
    combined = layers.Concatenate()([x_rna, x_protein])
    debug_tensor_shape(combined, "Combined Features for Transformer")
    combined = layers.Reshape((1, combined.shape[-1]))(combined)
    debug_tensor_shape(combined, "Reshaped Combined Features for Transformer")
    combined = layers.MultiHeadAttention(num_heads=8, key_dim=64)(combined, combined)
    debug_tensor_shape(combined, "Transformer Output")
    combined = layers.GlobalAveragePooling1D()(combined)
    debug_tensor_shape(combined, "Global Pooled Features")

    # Separate Path for Binding Prediction
    binding_path = layers.Dense(128, activation="relu")(combined)
    debug_tensor_shape(binding_path, "binding path output")
    binding_output = layers.Dense(1, activation="sigmoid", name="Binding_Prediction")(binding_path)

    # Separate Path for Protein Prediction
    protein_path = layers.Dense(128, activation="relu")(combined)
    debug_tensor_shape(protein_path, "Protein path output")
    protein_output = layers.Dense(17, activation="softmax", name="Protein_Prediction")(protein_path)

    # Model
    model = models.Model(inputs=[rna_input, protein_input], outputs=[binding_output, protein_output])
    model.compile(
        optimizer="adam",
        loss={
            "Binding_Prediction": "binary_crossentropy",
            "Protein_Prediction": "categorical_crossentropy",
        },
        metrics={
            "Binding_Prediction": "accuracy",
            "Protein_Prediction": "accuracy",
        },
    )
    return model

if __name__ == "__main__":
    # Define paths and parameters
    script_dir = os.getcwd()
    base_folder_path = os.path.join(script_dir, "Data", "datasets", "clip")

    # Process the data
    rna_data, protein_data, binding_labels, protein_labels, rna_folders = process_data(base_folder_path, total_samples)

    # Create indices for splitting
    indices = np.arange(len(rna_data))

    # Split into training and testing sets
    train_indices, test_indices = train_test_split(
        indices, test_size=0.2, stratify=protein_labels.argmax(axis=1), random_state=42
    )

    # Extract training and testing data
    rna_train, rna_test = rna_data[train_indices], rna_data[test_indices]
    protein_train, protein_test = protein_data[train_indices], protein_data[test_indices]
    binding_train, binding_test = binding_labels[train_indices], binding_labels[test_indices]
    protein_label_train, protein_label_test = protein_labels[train_indices], protein_labels[test_indices]

    # Create stratified train-validation split
    combined_keys = [(protein_label_train[i].argmax(), binding_train[i]) for i in range(len(protein_label_train))]

    train_sub_indices, val_indices = train_test_split(
        train_indices, test_size=0.2, stratify=combined_keys, random_state=42
    )

    # Prepare training and validation datasets
    train_rna, val_rna = rna_data[train_sub_indices], rna_data[val_indices]
    train_protein, val_protein = protein_data[train_sub_indices], protein_data[val_indices]
    train_binding, val_binding = binding_labels[train_sub_indices], binding_labels[val_indices]
    train_labels, val_labels = protein_labels[train_sub_indices], protein_labels[val_indices]

    # Model creation
    rna_input_shape = train_rna.shape[1:]
    protein_input_shape = train_protein.shape[1:]
    model = create_hybrid_model(rna_input_shape, protein_input_shape)

    # Model training
    model.fit(
        [train_rna, train_protein],
        {"Binding_Prediction": train_binding, "Protein_Prediction": train_labels},
        validation_data=([val_rna, val_protein], {"Binding_Prediction": val_binding, "Protein_Prediction": val_labels}),
        epochs=3,
        batch_size=32
    )

    # Predictions on the test set
    binding_predictions = model.predict([rna_test, protein_test])[0]  # Binding predictions
    protein_predictions = model.predict([rna_test, protein_test])[1]  # Protein class likelihoods

    # Prepare Metadata for Outputs
    rna_folders_test = [rna_folders[i] for i in test_indices]  # Get folder names for test data
    rna_numbers_test = np.arange(len(rna_test))  # Generate RNA numbers (you may have a specific mapping)
    binding_ground_truth = binding_test  # Ground truth for binding
    protein_ground_truth = np.argmax(protein_label_test, axis=1)  # Ground truth protein classes

    # Convert Predictions to Required Formats
    binding_pred_binary = (binding_predictions > 0.5).astype(int)  # Convert to binary
    protein_pred_classes = np.argmax(protein_predictions, axis=1)  # Predicted protein classes

    # First File: Core Predictions and Metadata
    core_output_data = pd.DataFrame({
        "Folder_Name": rna_folders_test,
        "RNA_Number": rna_numbers_test,
        "Binding_Ground_Truth": binding_ground_truth,
        "Binding_Prediction": binding_pred_binary.flatten(),
        "Protein_Ground_Class": protein_ground_truth,
        "Protein_Predicted_Class": protein_pred_classes
    })
    core_output_file = os.path.join(OUTPUT_DIR, "core_predictions.csv")
    core_output_data.to_csv(core_output_file, index=False)
    print(f"[INFO] Core predictions saved to {core_output_file}")

    # Second File: Protein Likelihoods
    likelihood_output_data = pd.DataFrame({
        "Folder_Name": rna_folders_test,
        "RNA_Number": rna_numbers_test,
    })

    # Add Likelihoods for Each Protein Class
    for class_index in range(protein_predictions.shape[1]):  # Loop over all 17 classes
        likelihood_output_data[f"Protein_Class_{class_index}_Likelihood"] = protein_predictions[:, class_index]

    likelihood_output_file = os.path.join(OUTPUT_DIR, "protein_likelihoods.csv")
    likelihood_output_data.to_csv(likelihood_output_file, index=False)
    print(f"[INFO] Protein likelihoods saved to {likelihood_output_file}")


As soon as we balanced the data ,with the same number of epochs (3), we see that in the first epoch the binding prediciton goes down to .55, but the protein prediction accuracy already starts higher than the last protein prediction accuracy from the previous model. This shows that balancing the data can dramatically improve the models learning by decreasing the overfitting. This overfit decrease of the binding prediciton accuracy made the total loss deacrese be simultaneous for both predictions increasing the protein prediction accuracy. Although this affected the binding  prediction accuracy. We also increeased the number of head from 4 to 8 to increase the sequence length it is clustering.

[DEBUG] Initial RNA Input Shape: (None, 101, 8)
[DEBUG] Initial Protein Input Shape: (None, 1003, 27)
Epoch 1/3
/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['RNA_Input', 'Protein_Input']. Received: the structure of inputs=('*', '*')
  warnings.warn(
/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
400/400 ━━━━━━━━━━━━━━━━━━━━ 101s 247ms/step - Binding_Prediction_accuracy: 0.4988 - Binding_Prediction_loss: 39.0924 - Protein_Prediction_accuracy: 0.3088 - Protein_Prediction_loss: 138.2883 - loss: 177.3809 - val_Binding_Prediction_accuracy: 0.5014 - val_Binding_Prediction_loss: 0.7165 - val_Protein_Prediction_accuracy: 0.7058 - val_Protein_Prediction_loss: 0.6788 - val_loss: 1.3953
Epoch 2/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 96s 240ms/step - Binding_Prediction_accuracy: 0.5030 - Binding_Prediction_loss: 0.7124 - Protein_Prediction_accuracy: 0.7602 - Protein_Prediction_loss: 0.5576 - loss: 1.2700 - val_Binding_Prediction_accuracy: 0.5183 - val_Binding_Prediction_loss: 0.6918 - val_Protein_Prediction_accuracy: 0.9434 - val_Protein_Prediction_loss: 0.2180 - val_loss: 0.9098
Epoch 3/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 96s 240ms/step - Binding_Prediction_accuracy: 0.5145 - Binding_Prediction_loss: 0.7058 - Protein_Prediction_accuracy: 0.9418 - Protein_Prediction_loss: 0.2183 - loss: 0.9241 - val_Binding_Prediction_accuracy: 0.5339 - val_Binding_Prediction_loss: 0.7032 - val_Protein_Prediction_accuracy: 0.9409 - val_Protein_Prediction_loss: 0.2423 - val_loss: 0.9454
 11/125 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step   
/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (32, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
[INFO] Core predictions saved to /Users/marcobenavides/repos/ML4FG/3D-RBP/Intermediate_steps/core_predictions.csv
[INFO] Protein likelihoods saved to /Users/marcobenavides/repos/ML4FG/3D-RBP/Intermediate_steps/protein_likelihoods.csv

Before and testing this same model with the data balanced the outcome was still very low for protein prediction below .2. By impleemnting a higher head to 8 it immediately bumped the results of the protein oprediction acccuarcy to up to .7 but no the binding prediction accuracy went down to .5

So i implemented postional embeddings hoping it woul rememebr the order of both sequences which are extremely important. and improve botha accuracies.

Same model as before but head of 8 and with positional embedding I also moddified the batch size by increasing it to 100 instead of 32 I thought this would help as it would allow for a more representative population because a 32 would 

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import glob
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict

DEBUG_MODE = False  # Set to False to disable debug messages
total_samples = 280000 #set the number of samples you want to test. Total number of samples is 1,079,034


# Set seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Output directory for intermediate files
OUTPUT_DIR = os.path.join(os.getcwd(), "Results")
os.makedirs(OUTPUT_DIR, exist_ok=True)

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, model_dim):
        super(PositionalEncoding, self).__init__()
        self.sequence_length = sequence_length
        self.model_dim = model_dim
        self.positional_encoding = self._generate_positional_encoding()

    def _generate_positional_encoding(self):
        positions = np.arange(self.sequence_length)[:, np.newaxis]
        dimensions = np.arange(self.model_dim)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (dimensions // 2)) / np.float32(self.model_dim))
        angle_rads = positions * angle_rates

        # Apply sin to even indices and cos to odd indices
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

    def call(self, x):
        return x + self.positional_encoding[:, :tf.shape(x)[1], :]

# Debugging Utility Function
def debug_tensor_shape(tensor, name):
    """Utility function to debug and print tensor shape."""
    if DEBUG_MODE:
        print(f"[DEBUG] {name} shape: {tensor.shape}")
    return tensor


# Helper Functions to Process RNA and Protein Data
def save_to_csv(data, filename):
    filepath = os.path.join(OUTPUT_DIR, filename)
    if isinstance(data, np.ndarray):
        pd.DataFrame(data).to_csv(filepath, index=False, header=False)
    elif isinstance(data, pd.DataFrame):
        data.to_csv(filepath, index=False)
    print(f"[INFO] Saved {filename} to {OUTPUT_DIR}")

def load_protein_data(protein_file_path):
    protein_data = pd.read_csv(protein_file_path)
    protein_label = protein_data["Protein_Label"].iloc[0]
    protein_data = protein_data.drop(columns=["Protein_Label"], errors="ignore")
    protein_data = protein_data.apply(pd.to_numeric, errors="coerce").dropna(axis=1).values
    return protein_data, int(protein_label)

def load_rna_data(rna_file_path):
    rna_data = pd.read_csv(rna_file_path)
    rna_data = rna_data.drop(columns=["Sequence_Name", "Position"], errors="ignore")
    rna_data = rna_data.apply(pd.to_numeric, errors="coerce").dropna(axis=1).values
    num_samples = rna_data.shape[0] // 101
    rna_samples = rna_data[:num_samples * 101].reshape(num_samples, 101, -1)
    rna_features = rna_samples[:, :, :-1]
    rna_binding = rna_samples[:, 0, -1]
    return rna_features, rna_binding

def process_data(base_folder_path, total_samples):
    rna_data_list = []
    protein_data_list = []
    binding_labels = []
    protein_labels = []
    rna_folders = []

    class_counts = defaultdict(int)
    max_per_class = total_samples // 34  # 17 classes × 2 (binding vs non-binding)

    for root, _, _ in os.walk(base_folder_path):  # Removed `dirs` and `files`
        if "Protein data" in root:
            protein_folder = root
            rna_folder = os.path.dirname(protein_folder)

            protein_file = glob.glob(os.path.join(protein_folder, "*_Concatenated_Encoding_Matrix.csv"))
            if not protein_file:
                continue
            protein_file = protein_file[0]

            rna_file = os.path.join(rna_folder, "RNA_sequences_combined_stacked.csv")
            if not os.path.exists(rna_file):
                continue

            protein_data, protein_label = load_protein_data(protein_file)
            rna_data, rna_binding = load_rna_data(rna_file)

            protein_label -= 1
            for i, rna in enumerate(rna_data):
                binding_label = rna_binding[i]

                # Enforce class balancing
                class_key = (protein_label, binding_label)
                if class_counts[class_key] >= max_per_class:
                    continue

                rna_data_list.append(rna)
                protein_data_list.append(np.expand_dims(protein_data, axis=0))
                binding_labels.append(binding_label)
                protein_labels.append(protein_label)
                rna_folders.append(rna_folder)
                class_counts[class_key] += 1

                # Stop if we have enough total samples
                if sum(class_counts.values()) >= total_samples:
                    break

    if not rna_data_list or not protein_data_list:
        raise ValueError("[ERROR] No RNA or Protein data was successfully loaded.")

    # Convert to arrays
    rna_data_array = np.array(rna_data_list, dtype=np.float32)
    protein_data_array = np.concatenate(protein_data_list, axis=0)
    binding_labels_array = np.array(binding_labels, dtype=np.float32)
    protein_labels_array = tf.keras.utils.to_categorical(protein_labels, num_classes=17)

    return rna_data_array, protein_data_array, binding_labels_array, protein_labels_array, rna_folders



def create_hybrid_model(rna_input_shape, protein_input_shape):
    # RNA Feature Extraction
    rna_input = layers.Input(shape=rna_input_shape, name="RNA_Input")
    print(f"[DEBUG] Initial RNA Input Shape: {rna_input.shape}")
    x_rna = layers.Conv1D(64, 3, activation="relu")(rna_input)
    x_rna = layers.MaxPooling1D(2)(x_rna)
    x_rna = layers.Conv1D(128, 3, activation="relu")(x_rna)
    x_rna = layers.MaxPooling1D(2)(x_rna)
    x_rna = layers.Flatten()(x_rna)

    # Protein Feature Extraction
    protein_input = layers.Input(shape=protein_input_shape, name="Protein_Input")
    print(f"[DEBUG] Initial Protein Input Shape: {protein_input.shape}")
    x_protein = layers.Conv1D(64, 3, activation="relu")(protein_input)
    x_protein = layers.MaxPooling1D(5)(x_protein)
    x_protein = layers.Conv1D(128, 3, activation="relu")(x_protein)
    x_protein = layers.MaxPooling1D(2)(x_protein)
    x_protein = layers.Flatten()(x_protein)

    # Combine Features
    combined = layers.Concatenate()([x_rna, x_protein])
    debug_tensor_shape(combined, "Combined Features for Transformer")
    combined = layers.Reshape((1, combined.shape[-1]))(combined)
    debug_tensor_shape(combined, "Reshaped Combined Features for Transformer")

    # Add Positional Encoding
    pos_enc_layer = PositionalEncoding(sequence_length=1, model_dim=combined.shape[-1])
    combined_with_pos_enc = pos_enc_layer(combined)

    # Transformer Block
    combined_with_pos_enc = layers.MultiHeadAttention(num_heads=32, key_dim=64)(combined_with_pos_enc, combined_with_pos_enc)
    debug_tensor_shape(combined_with_pos_enc, "Transformer Output")
    combined_with_pos_enc = layers.GlobalAveragePooling1D()(combined_with_pos_enc)
    debug_tensor_shape(combined_with_pos_enc, "Global Pooled Features")

    # Separate Path for Binding Prediction
    binding_path = layers.Dense(128, activation="relu")(combined_with_pos_enc)
    debug_tensor_shape(binding_path, "binding path output")
    binding_output = layers.Dense(1, activation="sigmoid", name="Binding_Prediction")(binding_path)

    # Separate Path for Protein Prediction
    protein_path = layers.Dense(128, activation="relu")(combined_with_pos_enc)
    debug_tensor_shape(protein_path, "Protein path output")
    protein_output = layers.Dense(17, activation="softmax", name="Protein_Prediction")(protein_path)

    # Model
    model = models.Model(inputs=[rna_input, protein_input], outputs=[binding_output, protein_output])
    model.compile(
        optimizer="adam",
        loss={
            "Binding_Prediction": "binary_crossentropy",
            "Protein_Prediction": "categorical_crossentropy",
        },
        metrics={
            "Binding_Prediction": "accuracy",
            "Protein_Prediction": "accuracy",
        },
    )
    return model


if __name__ == "__main__":
    # Define paths and parameters
    script_dir = os.getcwd()
    base_folder_path = os.path.join(script_dir, "Data", "datasets", "clip")

    # Process the data
    rna_data, protein_data, binding_labels, protein_labels, rna_folders = process_data(base_folder_path, total_samples)

    # Create indices for splitting
    indices = np.arange(len(rna_data))

    # Split into training and testing sets
    train_indices, test_indices = train_test_split(
        indices, test_size=0.2, stratify=protein_labels.argmax(axis=1), random_state=42
    )

    # Extract training and testing data
    rna_train, rna_test = rna_data[train_indices], rna_data[test_indices]
    protein_train, protein_test = protein_data[train_indices], protein_data[test_indices]
    binding_train, binding_test = binding_labels[train_indices], binding_labels[test_indices]
    protein_label_train, protein_label_test = protein_labels[train_indices], protein_labels[test_indices]

    # Create stratified train-validation split
    combined_keys = [(protein_label_train[i].argmax(), binding_train[i]) for i in range(len(protein_label_train))]

    train_sub_indices, val_indices = train_test_split(
        train_indices, test_size=0.2, stratify=combined_keys, random_state=42
    )

    # Prepare training and validation datasets
    train_rna, val_rna = rna_data[train_sub_indices], rna_data[val_indices]
    train_protein, val_protein = protein_data[train_sub_indices], protein_data[val_indices]
    train_binding, val_binding = binding_labels[train_sub_indices], binding_labels[val_indices]
    train_labels, val_labels = protein_labels[train_sub_indices], protein_labels[val_indices]

    # Model creation
    rna_input_shape = train_rna.shape[1:]
    protein_input_shape = train_protein.shape[1:]
    model = create_hybrid_model(rna_input_shape, protein_input_shape)

    # Model training
    model.fit(
        [train_rna, train_protein],
        {"Binding_Prediction": train_binding, "Protein_Prediction": train_labels},
        validation_data=([val_rna, val_protein], {"Binding_Prediction": val_binding, "Protein_Prediction": val_labels}),
        epochs=10,
        batch_size= 100
    )

    # Predictions on the test set
    binding_predictions = model.predict([rna_test, protein_test])[0]  # Binding predictions
    protein_predictions = model.predict([rna_test, protein_test])[1]  # Protein class likelihoods

    # Prepare Metadata for Outputs
    rna_folders_test = [rna_folders[i] for i in test_indices]  # Get folder names for test data
    rna_numbers_test = np.arange(len(rna_test))  # Generate RNA numbers (you may have a specific mapping)
    binding_ground_truth = binding_test  # Ground truth for binding
    protein_ground_truth = np.argmax(protein_label_test, axis=1)  # Ground truth protein classes

    # Convert Predictions to Required Formats
    binding_pred_binary = (binding_predictions > 0.5).astype(int)  # Convert to binary
    protein_pred_classes = np.argmax(protein_predictions, axis=1)  # Predicted protein classes

    # First File: Core Predictions and Metadata
    core_output_data = pd.DataFrame({
        "Folder_Name": rna_folders_test,
        "RNA_Number": rna_numbers_test,
        "Binding_Ground_Truth": binding_ground_truth,
        "Binding_Prediction": binding_pred_binary.flatten(),
        "Protein_Ground_Class": protein_ground_truth,
        "Protein_Predicted_Class": protein_pred_classes
    })
    core_output_file = os.path.join(OUTPUT_DIR, "core_predictions.csv")
    core_output_data.to_csv(core_output_file, index=False)
    print(f"[INFO] Core predictions saved to {core_output_file}")

    # Second File: Protein Likelihoods
    likelihood_output_data = pd.DataFrame({
        "Folder_Name": rna_folders_test,
        "RNA_Number": rna_numbers_test,
    })

    # Add Likelihoods for Each Protein Class
    for class_index in range(protein_predictions.shape[1]):  # Loop over all 17 classes
        likelihood_output_data[f"Protein_Class_{class_index}_Likelihood"] = protein_predictions[:, class_index]

    likelihood_output_file = os.path.join(OUTPUT_DIR, "protein_likelihoods.csv")
    likelihood_output_data.to_csv(likelihood_output_file, index=False)
    print(f"[INFO] Protein likelihoods saved to {likelihood_output_file}")


[DEBUG] Initial RNA Input Shape: (None, 101, 8)
[DEBUG] Initial Protein Input Shape: (None, 1003, 27)


2024-11-27 17:15:06.632003: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-11-27 17:15:06.632073: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2024-11-27 17:15:06.632085: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2024-11-27 17:15:06.632111: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-27 17:15:06.632123: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['RNA_Input', 'Protein_Input']. Received: the structure of inputs=('*', '*')
  warnings.warn(
/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 32, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
2024-11-27 17:15:30.549839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 132/1777 ━━━━━━━━━━━━━━━━━━━━ 1:15:49 3s/step - Binding_Prediction_accuracy: 0.5027 - Binding_Prediction_loss: 67.5589 - Protein_Prediction_accuracy: 0.0568 - Protein_Prediction_loss: 351.0433 - loss: 418.6022

KeyboardInterrupt: 

Results for previous model but with only 20,000 samples:

[DEBUG] Initial RNA Input Shape: (None, 101, 8)
[DEBUG] Initial Protein Input Shape: (None, 1003, 27)
Epoch 1/3
/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['RNA_Input', 'Protein_Input']. Received: the structure of inputs=('*', '*')
  warnings.warn(
/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
400/400 ━━━━━━━━━━━━━━━━━━━━ 97s 235ms/step - Binding_Prediction_accuracy: 0.4988 - Binding_Prediction_loss: 39.0924 - Protein_Prediction_accuracy: 0.3088 - Protein_Prediction_loss: 138.2883 - loss: 177.3809 - val_Binding_Prediction_accuracy: 0.5014 - val_Binding_Prediction_loss: 0.7165 - val_Protein_Prediction_accuracy: 0.7058 - val_Protein_Prediction_loss: 0.6788 - val_loss: 1.3953
Epoch 2/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 93s 233ms/step - Binding_Prediction_accuracy: 0.5030 - Binding_Prediction_loss: 0.7124 - Protein_Prediction_accuracy: 0.7602 - Protein_Prediction_loss: 0.5576 - loss: 1.2700 - val_Binding_Prediction_accuracy: 0.5183 - val_Binding_Prediction_loss: 0.6918 - val_Protein_Prediction_accuracy: 0.9434 - val_Protein_Prediction_loss: 0.2180 - val_loss: 0.9098
Epoch 3/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 92s 229ms/step - Binding_Prediction_accuracy: 0.5145 - Binding_Prediction_loss: 0.7058 - Protein_Prediction_accuracy: 0.9418 - Protein_Prediction_loss: 0.2183 - loss: 0.9241 - val_Binding_Prediction_accuracy: 0.5339 - val_Binding_Prediction_loss: 0.7032 - val_Protein_Prediction_accuracy: 0.9409 - val_Protein_Prediction_loss: 0.2423 - val_loss: 0.9454
 14/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step   
/Users/marcobenavides/miniconda3/envs/ML4FG/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (32, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[INFO] Core predictions saved to /Users/marcobenavides/repos/ML4FG/3D-RBP/Intermediate_steps/core_predictions.csv
[INFO] Protein likelihoods saved to /Users/marcobenavides/repos/ML4FG/3D-RBP/Intermediate_steps/protein_likelihoods.csv